In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/telecom-churn-case-study-hackathon-c40/data_dictionary.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/train.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/test.csv
/kaggle/input/telecom-churn-case-study-hackathon-c40/solution.csv


In [2]:
pd.set_option('display.max_rows', 30)

In [3]:
telcom_data = pd.read_csv('/kaggle/input/telecom-churn-case-study-hackathon-c40/train.csv');
telcom_data.set_index('id', inplace=True)

In [4]:
telcom_data.head()

,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,arpu_6,arpu_7,arpu_8,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
id,,,,,,,,,,,,,,,,,,,,,
0,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,31.277,87.009,7.527,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,0.000,122.787,42.953,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,60.806,103.176,0.000,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,156.362,205.260,111.095,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,240.708,128.191,101.565,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [5]:
telcom_data.shape

(69999, 171)

In [6]:
pd.set_option('display.max_rows', None)
telcom_data.dtypes
#pd.set_option('display.max_rows', 30)

circle_id                     int64
loc_og_t2o_mou              float64
std_og_t2o_mou              float64
loc_ic_t2o_mou              float64
last_date_of_month_6         object
last_date_of_month_7         object
last_date_of_month_8         object
arpu_6                      float64
arpu_7                      float64
arpu_8                      float64
onnet_mou_6                 float64
onnet_mou_7                 float64
onnet_mou_8                 float64
offnet_mou_6                float64
offnet_mou_7                float64
offnet_mou_8                float64
roam_ic_mou_6               float64
roam_ic_mou_7               float64
roam_ic_mou_8               float64
roam_og_mou_6               float64
roam_og_mou_7               float64
roam_og_mou_8               float64
loc_og_t2t_mou_6            float64
loc_og_t2t_mou_7            float64
loc_og_t2t_mou_8            float64
loc_og_t2m_mou_6            float64
loc_og_t2m_mou_7            float64
loc_og_t2m_mou_8            

In [7]:
# clean the data for safe checks like empty columns and empty rows...
def sanity_clean_data(telcom_data):
    #columns with all empty values to be removed
    telcom_data.dropna(axis=1, how='all', inplace=True)
    
    #columns with all same values to be removed as they dont provide any insight
    cols_nounique = [i for i in telcom_data.columns if telcom_data[i].nunique()==1]
    telcom_data.drop(columns=cols_nounique, inplace=True, axis=1)
    print(cols_nounique)
    
    #rows with all NAs to be removed
    telcom_data.dropna(axis=0,how='all',inplace=True)
    
sanity_clean_data(telcom_data)    

['circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou', 'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7', 'last_date_of_month_8', 'std_og_t2c_mou_6', 'std_og_t2c_mou_7', 'std_og_t2c_mou_8', 'std_ic_t2o_mou_6', 'std_ic_t2o_mou_7', 'std_ic_t2o_mou_8']


In [8]:
telcom_data.shape

(69999, 158)

In [9]:
telcom_data.head()

,arpu_6,arpu_7,arpu_8,onnet_mou_6,onnet_mou_7,onnet_mou_8,offnet_mou_6,offnet_mou_7,offnet_mou_8,roam_ic_mou_6,...,sachet_3g_7,sachet_3g_8,fb_user_6,fb_user_7,fb_user_8,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,churn_probability
id,,,,,,,,,,,,,,,,,,,,,
0,31.277,87.009,7.527,48.58,124.38,1.29,32.24,96.68,2.33,0.00,...,0,0,NaN,NaN,NaN,1958,0.0,0.0,0.0,0
1,0.000,122.787,42.953,0.00,0.00,0.00,0.00,25.99,30.89,0.00,...,0,0,NaN,1.0,NaN,710,0.0,0.0,0.0,0
2,60.806,103.176,0.000,0.53,15.93,0.00,53.99,82.05,0.00,0.00,...,0,0,NaN,NaN,NaN,882,0.0,0.0,0.0,0
3,156.362,205.260,111.095,7.26,16.01,0.00,68.76,78.48,50.23,0.00,...,0,0,NaN,NaN,NaN,982,0.0,0.0,0.0,0
4,240.708,128.191,101.565,21.28,4.83,6.13,56.99,38.11,9.63,53.64,...,1,0,1.0,1.0,1.0,647,0.0,0.0,0.0,0


In [10]:
telcom_data.dtypes
#since the data is only from year 2014, converting it to day of the year.
# to keep it extensible for any other years, multiplying it with days in year + day in year
def reform_dates(telcom_data):
    date_cols = ['date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8']
    for col_name in date_cols:
        telcom_data[col_name] = pd.to_datetime(telcom_data[col_name]).dt.year*366 + pd.to_datetime(telcom_data[col_name]).dt.dayofyear
    return telcom_data
telcom_data = reform_dates(telcom_data)
#DatetimeIndex(telcom_data['date_of_last_rech_6']).day

In [11]:
telcom_data.columns[telcom_data.isna().any()]

Index(['onnet_mou_6', 'onnet_mou_7', 'onnet_mou_8', 'offnet_mou_6',
       'offnet_mou_7', 'offnet_mou_8', 'roam_ic_mou_6', 'roam_ic_mou_7',
       'roam_ic_mou_8', 'roam_og_mou_6',
       ...
       'arpu_3g_8', 'arpu_2g_6', 'arpu_2g_7', 'arpu_2g_8', 'night_pck_user_6',
       'night_pck_user_7', 'night_pck_user_8', 'fb_user_6', 'fb_user_7',
       'fb_user_8'],
      dtype='object', length=114)

In [12]:
telcom_data['fb_user_6'].unique()

array([nan,  1.,  0.])

In [13]:
pd.set_option('display.max_rows', None)
telcom_data.dtypes

arpu_6                      float64
arpu_7                      float64
arpu_8                      float64
onnet_mou_6                 float64
onnet_mou_7                 float64
onnet_mou_8                 float64
offnet_mou_6                float64
offnet_mou_7                float64
offnet_mou_8                float64
roam_ic_mou_6               float64
roam_ic_mou_7               float64
roam_ic_mou_8               float64
roam_og_mou_6               float64
roam_og_mou_7               float64
roam_og_mou_8               float64
loc_og_t2t_mou_6            float64
loc_og_t2t_mou_7            float64
loc_og_t2t_mou_8            float64
loc_og_t2m_mou_6            float64
loc_og_t2m_mou_7            float64
loc_og_t2m_mou_8            float64
loc_og_t2f_mou_6            float64
loc_og_t2f_mou_7            float64
loc_og_t2f_mou_8            float64
loc_og_t2c_mou_6            float64
loc_og_t2c_mou_7            float64
loc_og_t2c_mou_8            float64
loc_og_mou_6                

In [14]:
# aug_vbc_3g, jul_vbc_3g, jun_vbc_3g are not following same naming standard, let's rename these columns
#telcom_data.rename(columns = {'aug_vbc_3g':'vbc_3g_8', 'jul_vbc_3g': 'vbc_3g_7', 'jun_vbc_3g': 'vbc_3g_6' }, inplace = True)

In [15]:
#Lets look at the data dictionary and classify numeric vs categorical columns
cat_col_prefix = ['date_of_last_rech_','date_of_last_rech_data_','night_pck_user_']
cat_cols = []
for i in range(6,9):
    for prefix in cat_col_prefix:
        cat_cols.append(prefix+str(i));
cat_cols

['date_of_last_rech_6',
 'date_of_last_rech_data_6',
 'night_pck_user_6',
 'date_of_last_rech_7',
 'date_of_last_rech_data_7',
 'night_pck_user_7',
 'date_of_last_rech_8',
 'date_of_last_rech_data_8',
 'night_pck_user_8']

In [16]:
num_cols = [i for i in telcom_data.columns if i not in cat_cols]

In [17]:
#Missing values imputation:
# impute all numeric cols with median and categorical ones with mode
#for col in num_cols:
#    telcom_data[col].fillna(telcom_data[col].median(), inplace=True)
def impute_missing_values(data):
    data[num_cols] = data.filter(num_cols).fillna(data.median(numeric_only=True).iloc[0])
    data[cat_cols] = data.filter(cat_cols).fillna(data.mode().iloc[0])
    return data;
telcom_data = impute_missing_values(telcom_data)

In [18]:
telcom_data.columns[telcom_data.isna().sum()>0]

Index([], dtype='object')

In [19]:
#telcom_data['night_pck_user_6']#, 'night_pck_user_7', 'night_pck_user_8']
telcom_data.shape

(69999, 158)

In [20]:
X = telcom_data.iloc[:,0:157]
y = telcom_data['churn_probability']

In [21]:
from sklearn.tree import DecisionTreeClassifier
dt_base = DecisionTreeClassifier(random_state=42, max_depth=4)

In [22]:
dt_base.fit(X, y)

DecisionTreeClassifier(max_depth=4, random_state=42)

In [23]:
y_train_pred = dt_base.predict(X)

In [24]:
from sklearn.metrics import confusion_matrix, accuracy_score
print(accuracy_score(y, y_train_pred))
confusion_matrix(y, y_train_pred)

0.9419991714167345


array([[61644,  1223],
       [ 2837,  4295]])

## 

In [25]:
test_data = pd.read_csv('/kaggle/input/telecom-churn-case-study-hackathon-c40/test.csv');
#test_data.set_index('id', inplace=True)
test_data = reform_dates(test_data)
test_data['churn_probability'] = 0
test_data = impute_missing_values(test_data)
y_test = test_data.pop('churn_probability')

In [26]:
X_test = test_data[X.columns]

In [27]:
test_data['churn_probability'] = dt_base.predict(X_test)

In [28]:
solution = test_data[['id', 'churn_probability']]

In [29]:
solution.set_index('id', inplace=True)

In [30]:
solution.head()

,churn_probability
id,
69999,0
70000,0
70001,1
70002,0
70003,0


In [31]:
solution.to_csv('solution.csv', header=True, index=True)

In [32]:
from IPython.display import HTML
import pandas as pd
import numpy as np

solution.to_csv('solution.csv')

def create_download_link(title = "Download CSV file", filename = "data.csv"):  
    html = '<a href={filename}>{title}</a>'
    html = html.format(title=title,filename=filename)
    return HTML(html)

# create a link to download the dataframe which was saved with .to_csv method
create_download_link(filename='solution.csv')